In [1]:
import pandas as pd
import numpy as np
import xml.etree.ElementTree as et
import pyodbc

In [64]:
def clean_df(df):

    df['Route Date'] = pd.to_datetime(
        df['Route Date'], format="%Y-%m-%d", errors='coerce')
    df['Disposal Date'] = pd.to_datetime(
        df['Disposal Date'], format="%Y-%m-%d", errors='coerce')
    df['Tip In Time'] = pd.to_datetime(
        df['Tip In Time'], infer_datetime_format=True, errors='coerce')
    df['Tip Out Time'] = pd.to_datetime(
        df['Tip Out Time'], infer_datetime_format=True, errors='coerce')
    
    df['Tip In Time'] = df['Tip In Time'].fillna(pd.to_datetime('2021-10-20 00:00:00',infer_datetime_format=True))      
    df['Tip Out Time'] = df['Tip Out Time'].fillna(pd.to_datetime('2021-10-20 00:00:00',infer_datetime_format=True))    
    
    df['Tip In Time'] = df['Tip In Time'].dt.time
    df['Tip Out Time'] = df['Tip Out Time'].dt.time
    return df

In [25]:
PATH = "C:\\Users\\User\\Desktop\\blob_storage\\tip_we\\tip_all.xml"
ns = "{urn:schemas-microsoft-com:office:spreadsheet}"
etree = et.parse(PATH)
eroot = etree.getroot()
rows_attr = f"{ns}Row"
data_attr = f"{ns}Data"
data = list()

for r in eroot.iter(tag=rows_attr):
    row = np.array([])

    for d in r.iter(tag=data_attr):
        row = np.append(row, d.text)

    row = np.array(row)
    data.append(row)

df = pd.DataFrame(data=data[1:], columns=data[0]).iloc[1:]

In [65]:
df_2 = clean_df(df)


In [72]:
# df_2  00:00:00


313.2090909090909

In [59]:
df_2['Tip Out Time']

# df_2['Tip In Time'] = df_2['Tip In Time'].dt.time
df_2['Tip Out Time'] = df_2['Tip In Time'].dt.time

# df_2[df_2['Tip Out Time'].lt('20211021')]

1        00:00:00
2        00:00:00
3        00:00:00
4        00:00:00
5        00:00:00
           ...   
86830    00:00:00
86831    00:00:00
86832    00:00:00
86833    00:00:00
86834    00:00:00
Name: Tip Out Time, Length: 86834, dtype: object

In [24]:
# df_1 = df[['Route Date','Disposal Date']].iloc[1:100]
df_2 = df[['Tip In Time', 'Tip Out Time']].iloc[1:100]
df_2


Tip In Time Tip Out Time
2        0 days       0 days
3        0 days       0 days
4        0 days       0 days
5        0 days       0 days
6        0 days       0 days
..          ...          ...
96       0 days       0 days
97       0 days       0 days
98       0 days       0 days
99       0 days       0 days
100      0 days       0 days

[99 rows x 2 columns]

In [ ]:
df_2

In [19]:
server = 'localhost'
database = 'STAGE'
username = 'SA'
password = 'Sydwaste123#'

cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server +
                      ';DATABASE='+database+';UID='+username+';PWD=' + password)

cursor = cnxn.cursor()

for index, row in df_2.iterrows():
    cursor.execute("INSERT INTO [experiment].[insert_date].[dateformat] \
    (dateformat_1,timeformat_1) values (?,?)",
                   row['Tip In Time'],
                   row['Tip Out Time']
                   )
cnxn.commit()
cursor.close()
cnxn.close()

Error: ('HYC00', '[HYC00] [Microsoft][ODBC SQL Server Driver]Optional feature not implemented (0) (SQLBindParameter)')

0.04124926340601061